<a href="https://colab.research.google.com/github/RajaOsama08/Stock-Market-Prediction/blob/main/Stock_market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
#import lib for K-NN
from sklearn.neighbors import KNeighborsRegressor
#import lib for XGBoost
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
#import lib for LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# import lib for random forest model
from sklearn.ensemble import RandomForestRegressor



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the file paths
file_paths = {
    'AAPL': '/content/drive/MyDrive/stock_data_set/AAPL.csv',
    'CSCO': '/content/drive/MyDrive/stock_data_set/CSCO.csv',
    'INTC': '/content/drive/MyDrive/stock_data_set/INTC.csv',
    'NVDA': '/content/drive/MyDrive/stock_data_set/NVDA.csv',
    'SAP': '/content/drive/MyDrive/stock_data_set/SAP.csv'
}

# Load the datasets into pandas DataFrames
dataframes = {symbol: pd.read_csv(path) for symbol, path in file_paths.items()}

# Display the first few rows of each DataFrame
for symbol, df in dataframes.items():
    print(f"First few rows of {symbol} DataFrame:")
    print(df.head())
    print("\n")

# Display the first few rows of the DataFrame
print(df.head())

First few rows of AAPL DataFrame:
         Date       Open       High        Low      Close  Adj Close  \
0  2014-06-02  22.641430  22.672501  22.232143  22.451786  19.859396   
1  2014-06-03  22.445000  22.812143  22.437500  22.769285  20.140236   
2  2014-06-04  22.765715  23.138929  22.718214  23.029285  20.370214   
3  2014-06-05  23.078571  23.191786  22.950357  23.119642  20.450136   
4  2014-06-06  23.210714  23.259287  23.016787  23.056070  20.393915   

      Volume  
0  369350800  
1  292709200  
2  335482000  
3  303805600  
4  349938400  


First few rows of CSCO DataFrame:
         Date       Open       High        Low      Close  Adj Close    Volume
0  2014-06-02  24.639999  24.799999  24.559999  24.780001  18.135729  20393300
1  2014-06-03  24.750000  24.820000  24.450001  24.680000  18.062546  23339100
2  2014-06-04  24.610001  24.650000  24.400000  24.500000  17.930799  20635100
3  2014-06-05  24.620001  24.740000  24.410000  24.700001  18.077187  23320300
4  2014-06-0

In [4]:
df1 = dataframes['AAPL']
df2 = dataframes['CSCO']
df3 = dataframes['INTC']
df4 = dataframes['NVDA']
df5 = dataframes['SAP']
print(df1)

            Date        Open        High         Low       Close   Adj Close  \
0     2014-06-02   22.641430   22.672501   22.232143   22.451786   19.859396   
1     2014-06-03   22.445000   22.812143   22.437500   22.769285   20.140236   
2     2014-06-04   22.765715   23.138929   22.718214   23.029285   20.370214   
3     2014-06-05   23.078571   23.191786   22.950357   23.119642   20.450136   
4     2014-06-06   23.210714   23.259287   23.016787   23.056070   20.393915   
...          ...         ...         ...         ...         ...         ...   
2513  2024-05-24  188.820007  190.580002  188.039993  189.979996  189.979996   
2514  2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2515  2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2516  2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2517  2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   

         Volume  
0     369350800  
1  

In [5]:
df1['Date'] = pd.to_datetime(df1['Date'])
print('first print',df1)
print('\n')
df1.set_index('Date', inplace=True)
print('second print',df1)
print('\n')

first print            Date        Open        High         Low       Close   Adj Close  \
0    2014-06-02   22.641430   22.672501   22.232143   22.451786   19.859396   
1    2014-06-03   22.445000   22.812143   22.437500   22.769285   20.140236   
2    2014-06-04   22.765715   23.138929   22.718214   23.029285   20.370214   
3    2014-06-05   23.078571   23.191786   22.950357   23.119642   20.450136   
4    2014-06-06   23.210714   23.259287   23.016787   23.056070   20.393915   
...         ...         ...         ...         ...         ...         ...   
2513 2024-05-24  188.820007  190.580002  188.039993  189.979996  189.979996   
2514 2024-05-28  191.509995  193.000000  189.100006  189.990005  189.990005   
2515 2024-05-29  189.610001  192.250000  189.509995  190.289993  190.289993   
2516 2024-05-30  190.759995  192.179993  190.630005  191.289993  191.289993   
2517 2024-05-31  191.440002  192.570007  189.910004  192.250000  192.250000   

         Volume  
0     369350800  
1  

**Handling missing values**

In [6]:
# Check for missing values
missing_values = df1.isnull().sum()
print(missing_values)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


**Feature Engineerig**



In [7]:
# Moving averages
# df1['MA10'] = df1['Adj Close'].rolling(window=10).mean()
# df1['MA50'] = df1['Adj Close'].rolling(window=50).mean()

# # Daily returns
# print('Daily returns')
# df1['Daily Return'] = df1['Adj Close'].pct_change()

# # Volatility
# print('volatility')
# df1['Volatility'] = df1['Daily Return'].rolling(window=10).std()

# print(df1)

In [8]:
df1.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2.518000e+03
mean,84.508962,85.410833,83.658418,84.571586,82.523540,1.264485e+08
std,58.391642,59.014316,57.809549,58.437072,58.895952,7.264746e+07
min,22.445000,22.672501,22.232143,22.451786,19.859396,2.404830e+07
25%,32.028126,32.265625,31.756876,31.984374,28.776829,7.673740e+07
50%,52.465000,52.986250,51.877500,52.358750,50.572672,1.069804e+08
75%,144.029999,146.444996,142.649994,144.742504,142.913410,1.550671e+08
max,198.020004,199.619995,197.000000,198.110001,197.589523,7.593852e+08


**Dropping NaN**

In [9]:
print(pd.isna(df1).sum().sum())
# Drop rows with NaNs
df1.dropna(inplace=True)

0


In [10]:
scaler = MinMaxScaler()

# Scaling the features except for the target variable (Adj Close)
df1[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(
    df1[['Open', 'High', 'Low', 'Close', 'Volume']])

**Setting data for model prepration**

In [11]:
# Define features and target variable
features = ['Open', 'High', 'Low', 'Close', 'Volume']
X = df1[features]
y = df1['Adj Close']

**Data Sampling for test-train**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [13]:
# K-NN model
knn_model = KNeighborsRegressor()

# Hyperparameter tuning using GridSearchCV
HyperParams = {'n_neighbors': range(1, 20)}
grid_search_cv = GridSearchCV(knn_model, HyperParams, cv=5, scoring='neg_mean_squared_error')
grid_search_cv.fit(X_train, y_train)

# K-NN model best_estimator
best_knn_est = grid_search_cv.best_estimator_

# Model Train on training data set
best_knn_est.fit(X_train, y_train)

# Performing prediction on test data set
y_pred = best_knn_est.predict(X_test)

# Model Evaluation
K_mse = mean_squared_error(y_test, y_pred)
rmse = K_mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best K-NN model: {best_knn_est}")
print(f"RMSE on test set: {rmse}")
print(f"MAE on test set: {mae}")
print(f"R² on test set: {r2}")

# Predicting the Adj Close price for the next day
next_day_features_KNN = X.iloc[-1].values.reshape(1, -1)
print('next_day_features_KNN',next_day_features_KNN)
next_day_prediction_KNN = best_knn_est.predict(next_day_features_KNN)
print(f"Predicted Adj Close price for the next day: {next_day_prediction_KNN[0]}")

Best K-NN model: KNeighborsRegressor(n_neighbors=1)
RMSE on test set: 6.587332101353741
MAE on test set: 4.235247946428572
R² on test set: 0.8797960967968486
next_day_features_KNN [[0.96252313 0.96015774 0.95943192 0.96663976 0.06950556]]
Predicted Adj Close price for the next day: 177.203201


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [14]:
# Step 4: XGBoost Model
xgb_model = XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Model Evaluation
XGB_mse = mean_squared_error(y_test, y_pred)
rmse = XGB_mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE on test set: {rmse}")
print(f"MAE on test set: {mae}")
print(f"R² on test set: {r2}")

# Predicting the Adj Close price for the next day
next_day_features_XGB = pd.DataFrame([X.iloc[-1]], columns=features)
next_day_features_scaled = scaler.transform(next_day_features_XGB)
next_day_prediction_XGB = xgb_model.predict(next_day_features_scaled)
print(f"Predicted Adj Close price for the next day: {next_day_prediction_XGB[0]}")

RMSE on test set: 6.4573561515605835
MAE on test set: 4.088681245626783
R² on test set: 0.8844928328713111
Predicted Adj Close price for the next day: 20.14866065979004


**Implementing LSTM models**

In [15]:
# Reshape input data for LSTM [samples, time steps, features]
X_train_lstm = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=1))

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5776.4141
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5734.0005
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4729.7549
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4024.2419
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4012.8555
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3433.6357
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3582.6443
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3074.1960
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2951.8123
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2956.2251
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2941.6340
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2677.5981
Epoch 13/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2648.0378
Epoch 14/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2593.2344
Epoch 15/100
63/63 ━━━━━━━━━━

**Implementing Random forest model**

In [16]:

# Predict using LSTM model
lstm_train_predictions = lstm_model.predict(X_train_lstm)
lstm_test_predictions = lstm_model.predict(X_test_lstm)

# Add LSTM predictions to the feature set
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()
X_train_rf['LSTM_Pred'] = lstm_train_predictions
X_test_rf['LSTM_Pred'] = lstm_test_predictions

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


RandomForestRegressor(random_state=42)

**Hybrid prediction LSTM-RF**

In [17]:
# # Predict using Random Forest model
# rf_train_predictions = rf_model.predict(X_train_rf)
# rf_test_predictions = rf_model.predict(X_test_rf)

# # Combine predictions (simple average in this case)
# train_predictions = (lstm_train_predictions.flatten() + rf_train_predictions) / 2
# test_predictions = (lstm_test_predictions.flatten() + rf_test_predictions) / 2

# train_mse = mean_squared_error(y_train, train_predictions)
# test_mse = mean_squared_error(y_test, test_predictions)


# test_r2 = r2_score(y_test, test_predictions)

# print(f'Test MSE: {test_mse}')
# print(f'Test R²: {test_r2}')

In [19]:
# Prepare data for LSTM
def dataset_lstm(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        # print('X data:',X)
        # doing predition for Adj Close
        y.append(data[i + time_steps, 0])
        # print('Y data:',y)
    return np.array(X), np.array(y)
print('value of X:',X)
print('value of Y:',y)

print('shape X:',X.shape)
print('shape Y:',y.shape)

print('*****************************************')
print('value X_train:',X_train)
print('value X_test:',X_test)
print('value y_train:',y_train)
print('value y_test:',y_test)

time_steps = 3
# sequence creation by adding target to features
features_lstm_seq = np.hstack([X_train, y_train.values.reshape(-1, 1)])
X_lstm, y_lstm = dataset_lstm(features_lstm_seq, time_steps)

print('**********LSTM-Features***********')
print('features_lstm_seq:',features_lstm_seq)
print('X_lstm:',X_lstm)
print('y_lstm:',y_lstm)

# Developing the LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, X_lstm.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_lstm, y_lstm, epochs=50, batch_size=32, verbose=0)

# Gettting feature by LSTM for training
lstm_feat_train = model.predict(X_lstm)

# Developing the Random Forest Model
rand_for = RandomForestRegressor(n_estimators=100)

# Randon Forest trained by the features of LSTM
rand_for.fit(lstm_feat_train, y_lstm)

# Prepare test data for LSTM
# sequence creation by adding target to features
lstm_test = np.hstack([X_test, y_test.values.reshape(-1, 1)])
X_lstm_test, y_lstm_test = dataset_lstm(lstm_test, time_steps)

# Extract test features from LSTM
lstm_test_pred = model.predict(X_lstm_test)

# Predict with Random Forest on LSTM features
rf_pred = rand_for.predict(lstm_test_pred)

# Evaluation
rf_rmse = np.sqrt(mean_squared_error(y_lstm_test, rf_pred))
rf_mae = mean_absolute_error(y_lstm_test, rf_pred)

rf_rmse, rf_mae

value of X:                 Open      High       Low     Close    Volume
Date                                                        
2014-06-02  0.001119  0.000000  0.000000  0.000000  0.469584
2014-06-03  0.000000  0.000789  0.001175  0.001807  0.365358
2014-06-04  0.001827  0.002636  0.002781  0.003288  0.423525
2014-06-05  0.003609  0.002935  0.004110  0.003802  0.380448
2014-06-06  0.004361  0.003316  0.004490  0.003440  0.443185
...              ...       ...       ...       ...       ...
2024-05-24  0.947601  0.948911  0.948732  0.953717  0.016654
2024-05-28  0.962922  0.962588  0.954797  0.953774  0.038393
2024-05-29  0.952100  0.958349  0.957143  0.955482  0.039464
2024-05-30  0.958650  0.957954  0.963552  0.961175  0.035221
2024-05-31  0.962523  0.960158  0.959432  0.966640  0.069506

[2518 rows x 5 columns]
value of Y: Date
2014-06-02     19.859396
2014-06-03     20.140236
2014-06-04     20.370214
2014-06-05     20.450136
2014-06-06     20.393915
                 ...    
202

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


(0.028865578258762323, 0.02017034172507761)

In [21]:
test_mse = mean_squared_error(y_lstm_test, rf_pred)

test_r2 = r2_score(y_lstm_test, rf_pred)

print(f'Test MSE: {test_mse}')
print(f'Test R²: {test_r2}')

Test MSE: 0.0008332216082127321
Test R²: 0.9270555545520022
